# ЗАДАНИЕ ПО ВЫЧМАТАМ. ВАРИАНТ 7.5

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sympy as sp

# Суточные колебания концентрации озона в атмосфере Рассмотрим простейшую математическую модель колебаний концентрации озона в ат- мосфере. Она описывается следующей неавтономной системой ОДУ: 

Параметры системы, начальные условия

In [2]:
omega = np.pi/43200  #c-1
k1, k2 = 1.63e-16, 4.66e-16
c3, c4 = 22.62, 7.601

In [95]:
t = np.arange(0, 432, 1)  # step of 1 second
def light_dep(w, c, m):  # m for fit in velocities
    res = np.arange(0)
    for _ in t:
        if np.sin(w*_) <= 0.0:
            continue
        elif round(np.exp(-c/np.sin(w*_))*m):  # not to use ky less than 2 score of k2y1y3
            res = np.append(res, round(np.exp(-c/np.sin(w*_))*10*m, 2))  # exponent >= -10m
    if not res.size>0:
        res = 0  # for the case, when it's darkness all the time
    return res

In [97]:
k3, k4 = light_dep(omega, c3, 1e15), light_dep(omega, c4, 1e11)  # e-16 and e-12 accordingly

In [5]:
y1_0, y2_0, y3_0 = 1e6, 3.7e16, 1e12  # см-3 

In [6]:
def kineq(y1, y2, y3):  
    dydt_1 = -k1*y1*y2 - k2*y1*y3 + 2*k3*y2 + k4*y3
    # dydt_2 = 0
    dydt_3 = k1*y1*y2 - k2*y1*y3 - k4*y3
    return {1:dydt_1, 3:dydt_3}

# Схема Розенброка CROS
Yn+1 = Yn + tauRek

[E - gamma* h* J]*w = f(x0 + a1*h, y0)

a1 = Rebeta = 1/2

y1 = y0 + h*Rew

Yn+1 = E - (1+j)/2*tau*f(Yn, t)*k = f*(Yn,t + tau/2), a = 1/2

beta = {(1+j)/2, (1-j)/2}
Re(beta) >= 1/2

Функция устойчивости
R(z) = [1+(1-beta)*z]/(1-beta*z) 

R(z) = 1/(1 - z + (z**2)/2)

# Матрица Якоби системы ОДУ в y0

In [27]:
vars = sp.symbols('y1 y2 y3') # Define variables
f = sp.sympify(['-k1*y1*y2 - k2*y1*y3 + 2*k3*y2 + k4*y3', '0', 'k1*y1*y2 - k2*y1*y3 - k4*y3']) # Define function
jacoby = sp.zeros(len(f),len(vars)) # Initialise null Jacobian matrix

In [28]:
# Fill Jacobian matrix with entries
for i, fi in enumerate(f):
     for j, s in enumerate(vars):
            jacoby[i,j] = sp.diff(fi, s)
jacoby

Matrix([
[-k1*y2 - k2*y3, -k1*y1 + 2*k3, -k2*y1 + k4],
[             0,             0,           0],
[ k1*y2 - k2*y3,         k1*y1, -k2*y1 - k4]])

In [29]:
jacobian = sp.Matrix.det(jacoby)  # det == 0
values = []
values.append(-k1*y2_0 - k2*y3_0)  #11
values.append(-k1*y1_0 + 2*k3)  #12
values.append(-k2*y1_0 + k4)  #13
values.append(0)  #21
values.append(0)  #22
values.append(0)  #23
values.append(k1*y2_0 - k2*y3_0)  #31
values.append(k1*y1_0)  #32
values.append(-k2*y1_0 - k4)  #33
values[1]

-1.629998e-10

In [30]:
j = sp.Matrix(3, 3, values)  # float, and array sometimes, jacoby specified in y0

# Составим уравнение схемы

In [114]:
gamma = (1+1j)/2
t0 = 0
h = 1
t_end = 100  # 172800
e = sp.eye(3)  # E matrix
matr = e - gamma*h*j
#matr[1]

In [184]:
e - j

Matrix([
[ 7.031466, 1.629998e-10,       4.6599e-10],
[        0,            1,                0],
[-6.030534,    -1.63e-10, 1.00000000046601]])

In [197]:
t = np.linspace(t0, t_end, 2*t_end+1)  # h=1, taken h/2
def rosenbrock(func): 
    def especially_for_row_num(row_num):  # with love
        kost = 0
        row_num = func(omega, c4, 1, t, row_num)
        assert row_num == 1 or 3
        w = np.arange(0)
        if row_num == 1:
            init_y = y1_0
        else:
            init_y = y3_0
        y_n = np.array(init_y)  # final decision Yn+1 = Yn + hRew
        for _ in t:
            if kost % 2 == 0:
                k3 = func(omega, c3, 1, t[kost])
                k4 = func(omega, c4, 1, t[kost])
                y = kineq(y1_0, y2_0, y3_0)[row_num]
                matr = e - gamma*h*j  # m*w = f(h, y0)
                if row_num!=1:
                    _m = matr[6] + matr[7] + matr[8]
                      # trying to realise matrix division
                else:
                    _m = matr[0] + matr[1] + matr[2]
            w = np.append(w, (y + _)/_m)
            y_n = np.append(y_n, init_y + h*sp.re((y + _)/_m))
            kost+=1
        #print(w)
        return y_n
    return especially_for_row_num 

In [198]:
@rosenbrock
def light_dep(w, c, m, t, row_num=None):  # deprecated function from the very beginning
    if not row_num:
        res = np.arange(0)
        if np.sin(w*t) <= 0.0:
            pass
        elif round(np.exp(-c/np.sin(w*t))*m):  
            res = np.append(res, round(np.exp(-c/np.sin(w*t))*10*m, 2)) 
        if not res.size>0:
            res = 0  
        return res
    else:
        return row_num

In [199]:
light_dep(1)

array([1000000.0, 39649.8804610220, 39649.9600726886, 39650.0396843551,
       39650.1192960216, 39650.1989076881, 39650.2785193546,
       39650.3581310211, 39650.4377426878, 39650.5173543541,
       39650.5969660206, 39650.6765776873, 39650.7561893538,
       39650.8358010201, 39650.9154126868, 39650.9950243533,
       39651.0746360198, 39651.1542476864, 39651.2338593528,
       39651.3134710193, 39651.3930826859, 39651.4726943524,
       39651.5523060188, 39651.6319176854, 39651.7115293519,
       39651.7911410184, 39651.8707526849, 39651.9503643514,
       39652.0299760179, 39652.1095876845, 39652.1891993509,
       39652.2688110174, 39652.3484226840, 39652.4280343505,
       39652.5076460169, 39652.5872576835, 39652.6668693500,
       39652.7464810165, 39652.8260926830, 39652.9057043495,
       39652.9853160160, 39653.0649276826, 39653.1445393491,
       39653.2241510155, 39653.3037626821, 39653.3833743486,
       39653.4629860151, 39653.5425976816, 39653.6222093481,
       39653.

# УЖЕ РУНГА

In [1]:
class Onestepmethod (object):
    def __init__(self, f, y0, t0, te, N, tol):
        self.f = f
        self.y0 = y0.astype(float)
        self.t0 = t0
        self.interval = [t0, te]
        self.grid = linspace(t0, te, N+2)
        self.h = (te-t0)/(N+1)
        self.N = N
        self.tol = tol
        self.m = len(y0)
        self.s = len(self.b)
        self.solution = None

    def step(self):
        ti, yi = self.grid[0], self.y0
        tim1 = ti
        yield np.hstack((array([ti]), yi))
        for ti in self.grid[1:]:
            yi = yi + self.h * self.phi(tim1, yi)
            tim1 = ti
            yield np.hstack((array([ti]), yi))

    def solve(self):
        self.solution = list(self.step())

    # To be implemented in a derived class
    def phi(self, tim1, yi):
        return 1

In [2]:
class RungeImplicit(Onestepmethod):
    def phi(self, t0, y0):
        M = 10
        stageDer = array(self.s*[self.f(t0,y0)])
        J = jacobian(self.f, t0, y0)
        stageVal = self.phi_solve(t0, y0, stageDer, J, M)
        return array([dot(self.b, stageVal.reshape(self.s,self.m)[:, j]) for j in range(self.m)])

    def phi_solve(self, t0, y0, initVal, J, M):
        JJ = eye(self.s*self.m)-self.h*np.kron(self.A, J)
        luFactor = linalg.lu_factor(JJ)
        for i in range(M):
            initVal, norm_d = self.phi_newtonstep(t0, y0, initVal, luFactor)
            if norm_d < self.tol:
                # print('Newton converged in {} steps'.format(i))
                break
            elif i == M-1:
                raise ValueError('The Newton iteration did not converge.')
        return initVal

    def phi_newtonstep(self, t0, y0, initVal, luFactor):
        d = linalg.lu_solve(luFactor, - self.F(initVal.flatten(), t0, y0))
        return initVal.flatten() + d, norm(d)

    def F(self, stageDer, t0, y0):
        stageDer_new = empty((self.s, self.m))
        for i in range(self.s):
            stageVal = y0 + array([self.h * dot(self.A[i, :],
                                  stageDer.reshape(self.s, self.m)[:, j]) for j in range(self.m)])
            stageDer_new[i, :] = self.f(t0 + self.c[i] * self.h, stageVal)
        return stageDer - stageDer_new.reshape(-1)

In [11]:
class Ordinate:
    def __init__(self, y1=y1_0, y2=y2_0, y3=y3_0):
        self.y1 = y1
        self.y2 = y2
        self.y3 = y3
    def __str__(self):
        return set(y1, y2, y3)

    
y = Ordinate()

In [45]:
t = np.linspace(0, 100, 201)
print(t[1])
def test(y ,t):
    dydt = 0.3*np.cos(y)
    return dydt
y0 = 1/2
y = odeint(test, y0, t)
#plt.plot(t, y)
#plt.show()
#y_1 = odeint(kineq, y1_0, t, axis=2)
#y_3 = odeint(kineq, y3_0, t, axis=2)

0.5
